``` python
# if you running at azure nootbook to prepare env
#!pip install lxml --ignore-installed lxml
#!pip install tushare==1.2.54 
```

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
from greenseer.configuration import do_global_configuration
do_global_configuration("config.properties")

In [4]:
from greenseer.dataset.china_dataset import fetch_default_targets

y = fetch_default_targets()
y.head(5)

,st
code,
603212,0
688126,0
688123,0
688088,0
688268,0


- 600519：茅台

In [5]:
from greenseer.dataset.china_dataset import fetch_multi_report
from logging.config import fileConfig
import numpy as np
fileConfig('logging_config.ini')
reports = fetch_multi_report(stock_ids=np.array(["600519"]))
reports.shape

(57, 242)

In [7]:
subjects = [r'\w*应付\w*|\w*应收\w*|存货|货币资金|\w*生物\w*']

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer
from greenseer.preprocessing.transformers import  regular_expression_column_filter, \
    pick_annual_report_china

prepare_pipeline = Pipeline([
        ('pick_annual_report', FunctionTransformer(func=pick_annual_report_china, validate=False)),
        ('pick_subject', FunctionTransformer(func=regular_expression_column_filter, validate=False,kw_args={"patterns": subjects})),
])

annual_reports = prepare_pipeline.fit_transform(reports)
for column in annual_reports.columns:
    print(column)



专项应付款(万元)
公益性生物资产(万元)
其他应付款(万元)
其他应收款(万元)
内部应付款(万元)
内部应收款(万元)
存货(万元)
应付保证金(万元)
应付债券(万元)
应付分保账款(万元)
应付利息(万元)
应付手续费及佣金(万元)
应付短期债券(万元)
应付票据(万元)
应付职工薪酬(万元)
应付股利(万元)
应付账款(万元)
应收保证金(万元)
应收保费(万元)
应收出口退税(万元)
应收分保合同准备金(万元)
应收分保账款(万元)
应收利息(万元)
应收票据(万元)
应收股利(万元)
应收补贴款(万元)
应收账款(万元)
生产性生物资产(万元)
货币资金(万元)
长期应付款(万元)
长期应收款(万元)
